# CNN

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, Input, BatchNormalization, Flatten
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import uuid
import os

# Thiết lập seed để tái lập
np.random.seed(42)
tf.random.set_seed(42)

# Cấu hình bật/tắt các bước
CONFIG = {
    'apply_gaussian_noise': True,  # Bật/tắt tăng cường dữ liệu bằng nhiễu Gaussian
    'normalize_features': False,   # Bật/tắt chuẩn hóa đặc trưng
    'log_transform_target': False, # Bật/tắt biến đổi log cho biến mục tiêu
    'save_model': True,            # Bật/tắt lưu mô hình
    'save_results': True,          # Bật/tắt lưu kết quả đánh giá
    'visualize_results': True,     # Bật/tắt trực quan hóa
    'noise_factor': 0.01,          # Độ lớn nhiễu Gaussian
    'num_noise_copies': 2,         # Số bản sao nhiễu
    'test_size': 0.15,             # Tỷ lệ tập test
    'k_folds': 3,                  # Số fold trong cross-validation
}

# Hàm tính các chỉ số đánh giá
def calculate_metrics(y_true, y_pred):
    mask = y_true > 0
    if np.sum(mask) == 0:
        return {
            'mae': np.nan, 'mse': np.nan, 'rmse': np.nan, 'r2': np.nan,
            'mmre': np.nan, 'mdmre': np.nan, 'pred25': np.nan
        }
    
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    mre = np.abs(y_true[mask] - y_pred[mask]) / y_true[mask]
    mmre = np.mean(mre)
    mdmre = np.median(mre)
    pred25 = np.mean(mre <= 0.25) * 100
    
    return {
        'mae': mae, 'mse': mse, 'rmse': rmse, 'r2': r2,
        'mmre': mmre, 'mdmre': mdmre, 'pred25': pred25
    }

# Hàm đọc và tiền xử lý dữ liệu
def load_and_preprocess_data(dataset_path, feature_columns, target_column, numeric_columns):
    df = pd.read_csv(dataset_path)
    
    # Kiểm tra cột tồn tại
    missing_cols = [col for col in feature_columns + [target_column] if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing columns in dataset: {missing_cols}")
    
    # Kiểm tra kiểu dữ liệu
    for col in numeric_columns:
        if not np.issubdtype(df[col].dtype, np.number):
            raise ValueError(f"Column {col} is not numeric")
    if not np.issubdtype(df[target_column].dtype, np.number):
        raise ValueError(f"Target column {target_column} is not numeric")
    
    # Kiểm tra NaN
    X = df[feature_columns].values
    y = df[target_column].values
    if np.any(np.isnan(X)) or np.any(np.isnan(y)):
        raise ValueError("Data contains NaN values")
    
    # Chuẩn hóa đặc trưng nếu bật
    if CONFIG['normalize_features']:
        X = (X - X.mean(axis=0)) / (X.std(axis=0) + 1e-8)
    
    # Biến đổi log cho mục tiêu nếu bật
    if CONFIG['log_transform_target']:
        y = np.log1p(y)
    
    return X, y, df

# Hàm tăng cường dữ liệu bằng nhiễu Gaussian
def augment_data(X, y, noise_factor=0.01, num_copies=2):
    if not CONFIG['apply_gaussian_noise']:
        return X, y
    
    X_aug = X.copy()
    y_aug = y.copy()
    
    for _ in range(num_copies):
        noise = np.random.normal(loc=0, scale=noise_factor, size=X.shape)
        X_noisy = X + noise
        X_aug = np.vstack((X_aug, X_noisy))
        y_aug = np.hstack((y_aug, y))
    
    return X_aug, y_aug

# Hàm xây dựng mô hình CNN
def build_cnn_model(input_shape, filters=8, l2_reg=0.01, dense_units=16, dropout_rate=0.3, learning_rate=0.001):
    l2_reg = max(l2_reg, 0.001)
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(filters, kernel_size=2, activation='relu', padding='same', kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        Conv1D(filters, kernel_size=2, activation='relu', padding='same', kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        Flatten(),
        Dense(dense_units, activation='relu', kernel_regularizer=l2(l2_reg)),
        Dropout(dropout_rate),
        Dense(1, activation='linear')
    ])
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=tf.keras.losses.Huber(), metrics=['mae'])
    return model

# Không gian siêu tham số
param_bounds = {
    'filters': (4, 16),
    'l2_reg': (0.001, 0.05),
    'dense_units': (8, 32),
    'dropout_rate': (0.2, 0.4),
    'learning_rate': (1e-4, 1e-2),
    'batch_size': (16, 32),
    'epochs': (50, 100)
}

# Hàm mã hóa & giải mã particle
def random_particle():
    return np.array([
        np.random.randint(param_bounds['filters'][0], param_bounds['filters'][1] + 1),
        np.random.uniform(param_bounds['l2_reg'][0], param_bounds['l2_reg'][1]),
        np.random.randint(param_bounds['dense_units'][0], param_bounds['dense_units'][1] + 1),
        np.random.uniform(param_bounds['dropout_rate'][0], param_bounds['dropout_rate'][1]),
        np.random.uniform(param_bounds['learning_rate'][0], param_bounds['learning_rate'][1]),
        np.random.randint(param_bounds['batch_size'][0], param_bounds['batch_size'][1] + 1),
        np.random.randint(param_bounds['epochs'][0], param_bounds['epochs'][1] + 1)
    ])

def decode_particle(particle):
    params = {
        'filters': int(particle[0]),
        'l2_reg': particle[1],
        'dense_units': int(particle[2]),
        'dropout_rate': particle[3],
        'learning_rate': particle[4],
        'batch_size': int(particle[5]),
        'epochs': int(particle[6])
    }
    params['l2_reg'] = max(params['l2_reg'], 0.001)
    return params

# Hàm fitness cho PSO
def fitness_function(particle, X_train, y_train):
    params = decode_particle(particle)
    model = build_cnn_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        **{k: v for k, v in params.items() if k not in ['batch_size', 'epochs']}
    )
    
    kf = KFold(n_splits=CONFIG['k_folds'], shuffle=True, random_state=42)
    rmse_scores = []
    
    for train_idx, val_idx in kf.split(X_train):
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]
        
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
        
        model.fit(X_tr, y_tr, epochs=params['epochs'], batch_size=params['batch_size'],
                  validation_split=0.2, verbose=0, callbacks=[early_stopping, reduce_lr])
        y_pred = model.predict(X_val, verbose=0).flatten()
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        rmse_scores.append(rmse)
    
    return np.mean(rmse_scores)

# Hàm chạy PSO
def run_pso_cnn(X_train, y_train, num_particles=15, max_iter=10):
    dim = len(param_bounds)
    bounds_array = np.array(list(param_bounds.values()))
    
    particles = [random_particle() for _ in range(num_particles)]
    velocities = [np.zeros(dim) for _ in range(num_particles)]
    p_best_positions = particles.copy()
    p_best_scores = [fitness_function(p, X_train, y_train) for p in particles]
    
    g_best_index = np.argmin(p_best_scores)
    g_best_position = p_best_positions[g_best_index]
    g_best_score = p_best_scores[g_best_index]
    
    w, c1, c2 = 0.5, 1.5, 1.5
    
    for iter in range(max_iter):
        print(f"\n🔁 Iteration {iter + 1}/{max_iter}")
        for i in range(num_particles):
            r1 = np.random.rand(dim)
            r2 = np.random.rand(dim)
            
            velocities[i] = (
                w * velocities[i]
                + c1 * r1 * (p_best_positions[i] - particles[i])
                + c2 * r2 * (g_best_position - particles[i])
            )
            
            particles[i] += velocities[i]
            particles[i] = np.clip(particles[i], bounds_array[:, 0], bounds_array[:, 1])
            particles[i][1] = max(particles[i][1], param_bounds['l2_reg'][0])
            
            score = fitness_function(particles[i], X_train, y_train)
            
            if score < p_best_scores[i]:
                p_best_scores[i] = score
                p_best_positions[i] = particles[i]
                
            if score < g_best_score:
                g_best_score = score
                g_best_position = particles[i]
                print(f"✅ Cập nhật g_best: Score = {g_best_score:.4f}")
    
    return g_best_position, g_best_score

# Hàm huấn luyện và đánh giá mô hình tối ưu
def train_and_evaluate(X_train, X_test, y_train, y_test, best_params):
    model = build_cnn_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        **{k: v for k, v in best_params.items() if k not in ['batch_size', 'epochs']}
    )
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
    
    kf = KFold(n_splits=CONFIG['k_folds'], shuffle=True, random_state=42)
    metrics_all = []
    history_all = {'loss': [], 'val_loss': []}
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
        print(f"\n📂 Fold {fold + 1}/{CONFIG['k_folds']}")
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]
        
        history = model.fit(
            X_tr, y_tr, epochs=best_params['epochs'], batch_size=best_params['batch_size'],
            validation_split=0.2, verbose=0, callbacks=[early_stopping, reduce_lr]
        )
        y_pred = model.predict(X_val, verbose=0).flatten()
        metrics = calculate_metrics(y_val, y_pred)
        metrics_all.append(metrics)
        print(f"✅ Fold {fold + 1} RMSE: {metrics['rmse']:.4f}")
        
        history_all['loss'].append(history.history['loss'])
        history_all['val_loss'].append(history.history['val_loss'])
    
    # Đánh giá trên tập test
    y_pred_test = model.predict(X_test, verbose=0).flatten()
    test_metrics = calculate_metrics(y_test, y_pred_test)
    
    # Tính trung bình lịch sử huấn luyện
    max_len = max(len(h) for h in history_all['loss'])
    loss_avg = np.mean([np.pad(h, (0, max_len - len(h)), 'constant', constant_values=np.nan) for h in history_all['loss']], axis=0)
    val_loss_avg = np.mean([np.pad(h, (0, max_len - len(h)), 'constant', constant_values=np.nan) for h in history_all['val_loss']], axis=0)
    
    return model, test_metrics, metrics_all, loss_avg, val_loss_avg, y_pred_test

# Hàm trực quan hóa kết quả
def visualize_results(dataset_name, y_test, y_pred, loss_avg, val_loss_avg):
    if not CONFIG['visualize_results']:
        return
    
    plt.figure(figsize=(15, 12))
    
    # Loss trung bình
    plt.subplot(2, 2, 1)
    plt.plot(loss_avg, label='Training Loss')
    plt.plot(val_loss_avg, label='Validation Loss')
    plt.title('Average Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Huber Loss')
    plt.legend()
    
    # Predicted vs Actual
    plt.subplot(2, 2, 2)
    plt.scatter(y_test, y_pred, alpha=0.5)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    plt.title('Predicted vs Actual Effort')
    plt.xlabel('Actual Effort')
    plt.ylabel('Predicted Effort')
    
    # Error Distribution
    errors = y_test - y_pred
    plt.subplot(2, 2, 3)
    sns.histplot(errors, kde=True)
    plt.title('Error Distribution')
    plt.xlabel('Prediction Error')
    plt.ylabel('Frequency')
    
    plt.tight_layout()
    os.makedirs('visualizations', exist_ok=True)
    plt.savefig(f'visualizations/{dataset_name}_results.png')
    plt.close()

# Hàm chính để chạy pipeline
def run_pipeline(dataset_config):
    dataset_name = dataset_config['name']
    print(f"\n🚀 Chạy pipeline cho dataset: {dataset_name}")
    
    # Đọc và tiền xử lý dữ liệu
    X, y, df = load_and_preprocess_data(
        dataset_config['path'],
        dataset_config['feature_columns'],
        dataset_config['target_column'],
        dataset_config['numeric_columns']
    )
    
    # Tăng cường dữ liệu
    X_aug, y_aug = augment_data(X, y, CONFIG['noise_factor'], CONFIG['num_noise_copies'])
    
    X_aug = X_aug.reshape(X_aug.shape[0], X_aug.shape[1], 1)
    
    # Chia tập train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X_aug, y_aug, test_size=CONFIG['test_size'], random_state=42
    )
    
    # Chạy PSO
    print("🔍 Tìm siêu tham số tối ưu bằng PSO...")
    best_particle, best_score = run_pso_cnn(X_train, y_train, num_particles=15, max_iter=10)
    best_params = decode_particle(best_particle)
    print(f"🏆 Siêu tham số tốt nhất: {best_params}")
    print(f"📉 Score tốt nhất: {best_score:.4f}")
    
    # Huấn luyện và đánh giá
    model, test_metrics, metrics_all, loss_avg, val_loss_avg, y_pred_test = train_and_evaluate(
        X_train, X_test, y_train, y_test, best_params
    )
    
    # In kết quả
    print("\n📈 Kết quả đánh giá trên tập test:")
    for metric, value in test_metrics.items():
        print(f"📌 {metric.upper():<7}: {value:.4f}" if not np.isnan(value) else f"📌 {metric.upper():<7}: NaN")
    
    # Lưu kết quả
    if CONFIG['save_results']:
        results_df = pd.DataFrame([test_metrics])
        os.makedirs('results', exist_ok=True)
        results_df.to_csv(f'results/{dataset_name}_results.csv', index=False)
        print(f"\nĐã lưu kết quả vào 'results/{dataset_name}_results.csv'")
    
    # Lưu mô hình
    if CONFIG['save_model']:
        os.makedirs('models', exist_ok=True)
        model.save(f'models/{dataset_name}_model.keras')
        print(f"Đã lưu mô hình vào 'models/{dataset_name}_model.keras'")
    
    # Trực quan hóa
    visualize_results(dataset_name, y_test, y_pred_test, loss_avg, val_loss_avg)
    print(f"Đã lưu hình ảnh trực quan hóa vào 'visualizations/{dataset_name}_results.png'")

# Cấu hình các dataset
datasets = [
    {
        'name': 'desharnais',
        'path': 'desharnais1.1_processed_corrected.csv',
        'feature_columns': [
            'TeamExp', 'ManagerExp', 'YearEnd', 'Length', 'Transactions', 'Entities',
            'Adjustment', 'PointsAjust', 'StartYear', 'ProjectDurationYears',
            'Transactions_Entities', 'Effort_PointsAjust', 'Effort_per_PointsAjust',
            'Transactions_per_Entities', 'Language_b\'1\'', 'Language_b\'2\'',
            'Language_b\'3\'', 'HighEffort', 'HighPointsAjust'
        ],
        'target_column': 'Effort',
        'numeric_columns': [
            'TeamExp', 'ManagerExp', 'YearEnd', 'Length', 'Transactions', 'Entities',
            'Adjustment', 'PointsAjust', 'StartYear', 'ProjectDurationYears',
            'Transactions_Entities', 'Effort_PointsAjust', 'Effort_per_PointsAjust',
            'Transactions_per_Entities', 'Language_b\'1\'', 'Language_b\'2\'',
            'Language_b\'3\'', 'HighEffort', 'HighPointsAjust'
        ]
    }

]

if __name__ == "__main__":
    for dataset in datasets:
        run_pipeline(dataset)


🚀 Chạy pipeline cho dataset: desharnais
🔍 Tìm siêu tham số tối ưu bằng PSO...

🔁 Iteration 1/10
✅ Cập nhật g_best: Score = 0.1415
✅ Cập nhật g_best: Score = 0.1396
✅ Cập nhật g_best: Score = 0.1393

🔁 Iteration 2/10
✅ Cập nhật g_best: Score = 0.1292
✅ Cập nhật g_best: Score = 0.0890

🔁 Iteration 3/10
✅ Cập nhật g_best: Score = 0.0839

🔁 Iteration 4/10
✅ Cập nhật g_best: Score = 0.0797

🔁 Iteration 5/10

🔁 Iteration 6/10

🔁 Iteration 7/10

🔁 Iteration 8/10

🔁 Iteration 9/10

🔁 Iteration 10/10
🏆 Siêu tham số tốt nhất: {'filters': 5, 'l2_reg': 0.001, 'dense_units': 24, 'dropout_rate': np.float64(0.17205939504736156), 'learning_rate': np.float64(0.006348859157055838), 'batch_size': 13, 'epochs': 107}
📉 Score tốt nhất: 0.0797

📂 Fold 1/3
✅ Fold 1 RMSE: 0.1536

📂 Fold 2/3
✅ Fold 2 RMSE: 0.2019

📂 Fold 3/3
✅ Fold 3 RMSE: 0.0938

📈 Kết quả đánh giá trên tập test:
📌 MAE    : 0.1781
📌 MSE    : 0.1376
📌 RMSE   : 0.3710
📌 R2     : 0.9029
📌 MMRE   : 0.4788
📌 MDMRE  : 0.0922
📌 PRED25 : 64.7059

Đã 

# MLP

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import os

# Thiết lập seed để tái lập
np.random.seed(42)
tf.random.set_seed(42)

# Cấu hình bật/tắt các bước
CONFIG = {
    'apply_gaussian_noise': True,  # Bật/tắt tăng cường dữ liệu bằng nhiễu Gaussian
    'normalize_features': False,   # Bật/tắt chuẩn hóa đặc trưng
    'log_transform_target': False, # Bật/tắt biến đổi log cho biến mục tiêu
    'save_model': True,            # Bật/tắt lưu mô hình
    'save_results': True,          # Bật/tắt lưu kết quả đánh giá
    'visualize_results': True,     # Bật/tắt trực quan hóa
    'noise_factor': 0.01,          # Độ lớn nhiễu Gaussian
    'num_noise_copies': 2,         # Số bản sao nhiễu
    'test_size': 0.15,             # Tỷ lệ tập test
    'k_folds': 3,                  # Số fold trong cross-validation
}

# Hàm tính các chỉ số đánh giá
def calculate_metrics(y_true, y_pred):
    mask = y_true > 0
    if np.sum(mask) == 0:
        return {
            'mae': np.nan, 'mse': np.nan, 'rmse': np.nan, 'r2': np.nan,
            'mmre': np.nan, 'mdmre': np.nan, 'pred25': np.nan
        }
    
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    mre = np.abs(y_true[mask] - y_pred[mask]) / y_true[mask]
    mmre = np.mean(mre)
    mdmre = np.median(mre)
    pred25 = np.mean(mre <= 0.25) * 100
    
    return {
        'mae': mae, 'mse': mse, 'rmse': rmse, 'r2': r2,
        'mmre': mmre, 'mdmre': mdmre, 'pred25': pred25
    }

# Hàm đọc và tiền xử lý dữ liệu
def load_and_preprocess_data(dataset_path, feature_columns, target_column, numeric_columns):
    df = pd.read_csv(dataset_path)
    
    # Kiểm tra cột tồn tại
    missing_cols = [col for col in feature_columns + [target_column] if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing columns in dataset: {missing_cols}")
    
    # Kiểm tra kiểu dữ liệu
    for col in numeric_columns:
        if not np.issubdtype(df[col].dtype, np.number):
            raise ValueError(f"Column {col} is not numeric")
    if not np.issubdtype(df[target_column].dtype, np.number):
        raise ValueError(f"Target column {target_column} is not numeric")
    
    # Kiểm tra NaN
    X = df[feature_columns].values
    y = df[target_column].values
    if np.any(np.isnan(X)) or np.any(np.isnan(y)):
        raise ValueError("Data contains NaN values")
    
    # Chuẩn hóa đặc trưng nếu bật
    if CONFIG['normalize_features']:
        X = (X - X.mean(axis=0)) / (X.std(axis=0) + 1e-8)
    
    # Biến đổi log cho mục tiêu nếu bật
    if CONFIG['log_transform_target']:
        y = np.log1p(y)
    
    return X, y, df

# Hàm tăng cường dữ liệu bằng nhiễu Gaussian
def augment_data(X, y, noise_factor=0.01, num_copies=2):
    if not CONFIG['apply_gaussian_noise']:
        return X, y
    
    X_aug = X.copy()
    y_aug = y.copy()
    
    for _ in range(num_copies):
        noise = np.random.normal(loc=0, scale=noise_factor, size=X.shape)
        X_noisy = X + noise
        X_aug = np.vstack((X_aug, X_noisy))
        y_aug = np.hstack((y_aug, y))
    
    return X_aug, y_aug

# Hàm xây dựng mô hình MLP
def build_mlp_model(input_dim, hidden_units1=16, hidden_units2=8, l2_reg=0.01, dropout_rate=0.3, learning_rate=0.001):
    l2_reg = max(l2_reg, 0.001)
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(hidden_units1, activation='relu', kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        Dropout(dropout_rate),
        Dense(hidden_units2, activation='relu', kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        Dropout(dropout_rate),
        Dense(1, activation='linear')
    ])
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=tf.keras.losses.Huber(), metrics=['mae'])
    return model

# Không gian siêu tham số cho MLP
param_bounds = {
    'hidden_units1': (8, 32),
    'hidden_units2': (4, 16),
    'l2_reg': (0.001, 0.05),
    'dropout_rate': (0.2, 0.4),
    'learning_rate': (1e-4, 1e-2),
    'batch_size': (16, 32),
    'epochs': (50, 100)
}

# Hàm mã hóa & giải mã particle
def random_particle():
    return np.array([
        np.random.randint(param_bounds['hidden_units1'][0], param_bounds['hidden_units1'][1] + 1),
        np.random.randint(param_bounds['hidden_units2'][0], param_bounds['hidden_units2'][1] + 1),
        np.random.uniform(param_bounds['l2_reg'][0], param_bounds['l2_reg'][1]),
        np.random.uniform(param_bounds['dropout_rate'][0], param_bounds['dropout_rate'][1]),
        np.random.uniform(param_bounds['learning_rate'][0], param_bounds['learning_rate'][1]),
        np.random.randint(param_bounds['batch_size'][0], param_bounds['batch_size'][1] + 1),
        np.random.randint(param_bounds['epochs'][0], param_bounds['epochs'][1] + 1)
    ])

def decode_particle(particle):
    params = {
        'hidden_units1': int(particle[0]),
        'hidden_units2': int(particle[1]),
        'l2_reg': particle[2],
        'dropout_rate': particle[3],
        'learning_rate': particle[4],
        'batch_size': int(particle[5]),
        'epochs': int(particle[6])
    }
    params['l2_reg'] = max(params['l2_reg'], 0.001)
    return params

# Hàm fitness cho PSO
def fitness_function(particle, X_train, y_train):
    params = decode_particle(particle)
    model = build_mlp_model(
        input_dim=X_train.shape[1],
        **{k: v for k, v in params.items() if k not in ['batch_size', 'epochs']}
    )
    
    kf = KFold(n_splits=CONFIG['k_folds'], shuffle=True, random_state=42)
    rmse_scores = []
    
    for train_idx, val_idx in kf.split(X_train):
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]
        
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
        
        model.fit(X_tr, y_tr, epochs=params['epochs'], batch_size=params['batch_size'],
                  validation_split=0.2, verbose=0, callbacks=[early_stopping, reduce_lr])
        y_pred = model.predict(X_val, verbose=0).flatten()
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        rmse_scores.append(rmse)
    
    return np.mean(rmse_scores)

# Hàm chạy PSO
def run_pso_mlp(X_train, y_train, num_particles=15, max_iter=10):
    dim = len(param_bounds)
    bounds_array = np.array(list(param_bounds.values()))
    
    particles = [random_particle() for _ in range(num_particles)]
    velocities = [np.zeros(dim) for _ in range(num_particles)]
    p_best_positions = particles.copy()
    p_best_scores = [fitness_function(p, X_train, y_train) for p in particles]
    
    g_best_index = np.argmin(p_best_scores)
    g_best_position = p_best_positions[g_best_index]
    g_best_score = p_best_scores[g_best_index]
    
    w, c1, c2 = 0.5, 1.5, 1.5
    
    for iter in range(max_iter):
        print(f"\n🔁 Iteration {iter + 1}/{max_iter}")
        for i in range(num_particles):
            r1 = np.random.rand(dim)
            r2 = np.random.rand(dim)
            
            velocities[i] = (
                w * velocities[i]
                + c1 * r1 * (p_best_positions[i] - particles[i])
                + c2 * r2 * (g_best_position - particles[i])
            )
            
            particles[i] += velocities[i]
            particles[i] = np.clip(particles[i], bounds_array[:, 0], bounds_array[:, 1])
            particles[i][2] = max(particles[i][2], param_bounds['l2_reg'][0])
            
            score = fitness_function(particles[i], X_train, y_train)
            
            if score < p_best_scores[i]:
                p_best_scores[i] = score
                p_best_positions[i] = particles[i]
                
            if score < g_best_score:
                g_best_score = score
                g_best_position = particles[i]
                print(f"✅ Cập nhật g_best: Score = {g_best_score:.4f}")
    
    return g_best_position, g_best_score

# Hàm huấn luyện và đánh giá mô hình tối ưu
def train_and_evaluate(X_train, X_test, y_train, y_test, best_params):
    model = build_mlp_model(
        input_dim=X_train.shape[1],
        **{k: v for k, v in best_params.items() if k not in ['batch_size', 'epochs']}
    )
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
    
    kf = KFold(n_splits=CONFIG['k_folds'], shuffle=True, random_state=42)
    metrics_all = []
    history_all = {'loss': [], 'val_loss': []}
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
        print(f"\n📂 Fold {fold + 1}/{CONFIG['k_folds']}")
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]
        
        history = model.fit(
            X_tr, y_tr, epochs=best_params['epochs'], batch_size=best_params['batch_size'],
            validation_split=0.2, verbose=0, callbacks=[early_stopping, reduce_lr]
        )
        y_pred = model.predict(X_val, verbose=0).flatten()
        metrics = calculate_metrics(y_val, y_pred)
        metrics_all.append(metrics)
        print(f"✅ Fold {fold + 1} RMSE: {metrics['rmse']:.4f}")
        
        history_all['loss'].append(history.history['loss'])
        history_all['val_loss'].append(history.history['val_loss'])
    
    # Đánh giá trên tập test
    y_pred_test = model.predict(X_test, verbose=0).flatten()
    test_metrics = calculate_metrics(y_test, y_pred_test)
    
    # Tính trung bình lịch sử huấn luyện
    max_len = max(len(h) for h in history_all['loss'])
    loss_avg = np.mean([np.pad(h, (0, max_len - len(h)), 'constant', constant_values=np.nan) for h in history_all['loss']], axis=0)
    val_loss_avg = np.mean([np.pad(h, (0, max_len - len(h)), 'constant', constant_values=np.nan) for h in history_all['val_loss']], axis=0)
    
    return model, test_metrics, metrics_all, loss_avg, val_loss_avg, y_pred_test

# Hàm trực quan hóa kết quả
def visualize_results(dataset_name, y_test, y_pred, loss_avg, val_loss_avg):
    if not CONFIG['visualize_results']:
        return
    
    plt.figure(figsize=(15, 12))
    
    # Loss trung bình
    plt.subplot(2, 2, 1)
    plt.plot(loss_avg, label='Training Loss')
    plt.plot(val_loss_avg, label='Validation Loss')
    plt.title('Average Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Huber Loss')
    plt.legend()
    
    # Predicted vs Actual
    plt.subplot(2, 2, 2)
    plt.scatter(y_test, y_pred, alpha=0.5)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    plt.title('Predicted vs Actual Effort')
    plt.xlabel('Actual Effort')
    plt.ylabel('Predicted Effort')
    
    # Error Distribution
    errors = y_test - y_pred
    plt.subplot(2, 2, 3)
    sns.histplot(errors, kde=True)
    plt.title('Error Distribution')
    plt.xlabel('Prediction Error')
    plt.ylabel('Frequency')
    
    plt.tight_layout()
    os.makedirs('visualizations', exist_ok=True)
    plt.savefig(f'visualizations/{dataset_name}_mlp_results.png')
    plt.close()

# Hàm chính để chạy pipeline
def run_pipeline(dataset_config):
    dataset_name = dataset_config['name']
    print(f"\n🚀 Chạy pipeline cho dataset: {dataset_name}")
    
    # Đọc và tiền xử lý dữ liệu
    X, y, df = load_and_preprocess_data(
        dataset_config['path'],
        dataset_config['feature_columns'],
        dataset_config['target_column'],
        dataset_config['numeric_columns']
    )
    
    # Tăng cường dữ liệu
    X_aug, y_aug = augment_data(X, y, CONFIG['noise_factor'], CONFIG['num_noise_copies'])
    
    # Chia tập train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X_aug, y_aug, test_size=CONFIG['test_size'], random_state=42
    )
    
    # Chạy PSO
    print("🔍 Tìm siêu tham số tối ưu bằng PSO...")
    best_particle, best_score = run_pso_mlp(X_train, y_train, num_particles=15, max_iter=10)
    best_params = decode_particle(best_particle)
    print(f"🏆 Siêu tham số tốt nhất: {best_params}")
    print(f"📉 Score tốt nhất: {best_score:.4f}")
    
    # Huấn luyện và đánh giá
    model, test_metrics, metrics_all, loss_avg, val_loss_avg, y_pred_test = train_and_evaluate(
        X_train, X_test, y_train, y_test, best_params
    )
    
    # In kết quả
    print("\n📈 Kết quả đánh giá trên tập test:")
    for metric, value in test_metrics.items():
        print(f"📌 {metric.upper():<7}: {value:.4f}" if not np.isnan(value) else f"📌 {metric.upper():<7}: NaN")
    
    # Lưu kết quả
    if CONFIG['save_results']:
        results_df = pd.DataFrame([test_metrics])
        os.makedirs('results', exist_ok=True)
        results_df.to_csv(f'results/{dataset_name}_mlp_results.csv', index=False)
        print(f"\nĐã lưu kết quả vào 'results/{dataset_name}_mlp_results.csv'")
    
    # Lưu mô hình
    if CONFIG['save_model']:
        os.makedirs('models', exist_ok=True)
        model.save(f'models/{dataset_name}_mlp_model.keras')
        print(f"Đã lưu mô hình vào 'models/{dataset_name}_mlp_model.keras'")
    
    # Trực quan hóa
    visualize_results(dataset_name, y_test, y_pred_test, loss_avg, val_loss_avg)
    print(f"Đã lưu hình ảnh trực quan hóa vào 'visualizations/{dataset_name}_mlp_results.png'")

# Cấu hình các dataset
datasets = [
    {
        'name': 'desharnais',
        'path': 'desharnais1.1_processed_corrected.csv',
        'feature_columns': [
            'TeamExp', 'ManagerExp', 'YearEnd', 'Length', 'Transactions', 'Entities',
            'Adjustment', 'PointsAjust', 'StartYear', 'ProjectDurationYears',
            'Transactions_Entities', 'Effort_PointsAjust', 'Effort_per_PointsAjust',
            'Transactions_per_Entities', 'Language_b\'1\'', 'Language_b\'2\'',
            'Language_b\'3\'', 'HighEffort', 'HighPointsAjust'
        ],
        'target_column': 'Effort',
        'numeric_columns': [
            'TeamExp', 'ManagerExp', 'YearEnd', 'Length', 'Transactions', 'Entities',
            'Adjustment', 'PointsAjust', 'StartYear', 'ProjectDurationYears',
            'Transactions_Entities', 'Effort_PointsAjust', 'Effort_per_PointsAjust',
            'Transactions_per_Entities', 'Language_b\'1\'', 'Language_b\'2\'',
            'Language_b\'3\'', 'HighEffort', 'HighPointsAjust'
        ]
    }
]

# Chạy pipeline cho từng dataset
if __name__ == "__main__":
    for dataset in datasets:
        run_pipeline(dataset)


🚀 Chạy pipeline cho dataset: desharnais
🔍 Tìm siêu tham số tối ưu bằng PSO...

🔁 Iteration 1/10
✅ Cập nhật g_best: Score = 0.1859

🔁 Iteration 2/10

🔁 Iteration 3/10
✅ Cập nhật g_best: Score = 0.1592

🔁 Iteration 4/10

🔁 Iteration 5/10

🔁 Iteration 6/10

🔁 Iteration 7/10

🔁 Iteration 8/10

🔁 Iteration 9/10

🔁 Iteration 10/10
🏆 Siêu tham số tốt nhất: {'hidden_units1': 35, 'hidden_units2': 11, 'l2_reg': np.float64(0.045730786735993637), 'dropout_rate': np.float64(0.243068412372354), 'learning_rate': np.float64(0.0044283623738321475), 'batch_size': 18, 'epochs': 71}
📉 Score tốt nhất: 0.1592

📂 Fold 1/3
✅ Fold 1 RMSE: 0.3559

📂 Fold 2/3
✅ Fold 2 RMSE: 0.0884

📂 Fold 3/3
✅ Fold 3 RMSE: 0.0669

📈 Kết quả đánh giá trên tập test:
📌 MAE    : 0.0634
📌 MSE    : 0.0058
📌 RMSE   : 0.0760
📌 R2     : 0.9959
📌 MMRE   : 0.5409
📌 MDMRE  : 0.0520
📌 PRED25 : 88.2353

Đã lưu kết quả vào 'results/desharnais_mlp_results.csv'
Đã lưu mô hình vào 'models/desharnais_mlp_model.keras'
Đã lưu hình ảnh trực quan hó

# LSTM

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import os

# Thiết lập seed để tái lập
np.random.seed(42)
tf.random.set_seed(42)

# Cấu hình bật/tắt các bước
CONFIG = {
    'apply_gaussian_noise': True,  # Bật/tắt tăng cường dữ liệu bằng nhiễu Gaussian
    'normalize_features': False,   # Bật/tắt chuẩn hóa đặc trưng
    'log_transform_target': False, # Bật/tắt biến đổi log cho biến mục tiêu
    'save_model': True,            # Bật/tắt lưu mô hình
    'save_results': True,          # Bật/tắt lưu kết quả đánh giá
    'visualize_results': True,     # Bật/tắt trực quan hóa
    'noise_factor': 0.01,          # Độ lớn nhiễu Gaussian
    'num_noise_copies': 2,         # Số bản sao nhiễu
    'test_size': 0.15,             # Tỷ lệ tập test
    'k_folds': 3,                  # Số fold trong cross-validation
    'timesteps': 1                 # Số bước thời gian cho LSTM
}

# Hàm tính các chỉ số đánh giá
def calculate_metrics(y_true, y_pred):
    mask = y_true > 0
    if np.sum(mask) == 0:
        return {
            'mae': np.nan, 'mse': np.nan, 'rmse': np.nan, 'r2': np.nan,
            'mmre': np.nan, 'mdmre': np.nan, 'pred25': np.nan
        }
    
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    mre = np.abs(y_true[mask] - y_pred[mask]) / y_true[mask]
    mmre = np.mean(mre)
    mdmre = np.median(mre)
    pred25 = np.mean(mre <= 0.25) * 100
    
    return {
        'mae': mae, 'mse': mse, 'rmse': rmse, 'r2': r2,
        'mmre': mmre, 'mdmre': mdmre, 'pred25': pred25
    }

# Hàm đọc và tiền xử lý dữ liệu
def load_and_preprocess_data(dataset_path, feature_columns, target_column, numeric_columns):
    df = pd.read_csv(dataset_path)
    
    # Kiểm tra cột tồn tại
    missing_cols = [col for col in feature_columns + [target_column] if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing columns in dataset: {missing_cols}")
    
    # Kiểm tra kiểu dữ liệu
    for col in numeric_columns:
        if not np.issubdtype(df[col].dtype, np.number):
            raise ValueError(f"Column {col} is not numeric")
    if not np.issubdtype(df[target_column].dtype, np.number):
        raise ValueError(f"Target column {target_column} is not numeric")
    
    # Kiểm tra NaN
    X = df[feature_columns].values
    y = df[target_column].values
    if np.any(np.isnan(X)) or np.any(np.isnan(y)):
        raise ValueError("Data contains NaN values")
    
    # Chuẩn hóa đặc trưng nếu bật
    if CONFIG['normalize_features']:
        X = (X - X.mean(axis=0)) / (X.std(axis=0) + 1e-8)
    
    # Biến đổi log cho mục tiêu nếu bật
    if CONFIG['log_transform_target']:
        y = np.log1p(y)
    
    return X, y, df

# Hàm tăng cường dữ liệu bằng nhiễu Gaussian
def augment_data(X, y, noise_factor=0.01, num_copies=2):
    if not CONFIG['apply_gaussian_noise']:
        return X, y
    
    X_aug = X.copy()
    y_aug = y.copy()
    
    for _ in range(num_copies):
        noise = np.random.normal(loc=0, scale=noise_factor, size=X.shape)
        X_noisy = X + noise
        X_aug = np.vstack((X_aug, X_noisy))
        y_aug = np.hstack((y_aug, y))
    
    return X_aug, y_aug

# Hàm reshape dữ liệu cho LSTM
def reshape_for_lstm(X, timesteps=1):
    # Reshape dữ liệu thành [samples, timesteps, features]
    n_samples, n_features = X.shape
    n_timesteps = timesteps
    n_new_samples = n_samples // n_timesteps
    X_reshaped = X[:n_new_samples * n_timesteps].reshape(n_new_samples, n_timesteps, n_features)
    return X_reshaped

# Hàm xây dựng mô hình LSTM
def build_lstm_model(input_shape, lstm_units=16, dense_units=8, l2_reg=0.01, dropout_rate=0.3, learning_rate=0.001):
    l2_reg = max(l2_reg, 0.001)
    model = Sequential([
        Input(shape=input_shape),
        LSTM(lstm_units, activation='tanh', recurrent_activation='sigmoid', return_sequences=False,
             kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        Dropout(dropout_rate),
        Dense(dense_units, activation='relu', kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        Dropout(dropout_rate),
        Dense(1, activation='linear')
    ])
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=tf.keras.losses.Huber(), metrics=['mae'])
    return model

# Không gian siêu tham số cho LSTM
param_bounds = {
    'lstm_units': (8, 32),
    'dense_units': (4, 16),
    'l2_reg': (0.001, 0.05),
    'dropout_rate': (0.2, 0.4),
    'learning_rate': (1e-4, 1e-2),
    'batch_size': (16, 32),
    'epochs': (50, 100)
}

# Hàm mã hóa & giải mã particle
def random_particle():
    return np.array([
        np.random.randint(param_bounds['lstm_units'][0], param_bounds['lstm_units'][1] + 1),
        np.random.randint(param_bounds['dense_units'][0], param_bounds['dense_units'][1] + 1),
        np.random.uniform(param_bounds['l2_reg'][0], param_bounds['l2_reg'][1]),
        np.random.uniform(param_bounds['dropout_rate'][0], param_bounds['dropout_rate'][1]),
        np.random.uniform(param_bounds['learning_rate'][0], param_bounds['learning_rate'][1]),
        np.random.randint(param_bounds['batch_size'][0], param_bounds['batch_size'][1] + 1),
        np.random.randint(param_bounds['epochs'][0], param_bounds['epochs'][1] + 1)
    ])

def decode_particle(particle):
    params = {
        'lstm_units': int(particle[0]),
        'dense_units': int(particle[1]),
        'l2_reg': particle[2],
        'dropout_rate': particle[3],
        'learning_rate': particle[4],
        'batch_size': int(particle[5]),
        'epochs': int(particle[6])
    }
    params['l2_reg'] = max(params['l2_reg'], 0.001)
    return params

# Hàm fitness cho PSO
def fitness_function(particle, X_train, y_train):
    params = decode_particle(particle)
    model = build_lstm_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        **{k: v for k, v in params.items() if k not in ['batch_size', 'epochs']}
    )
    
    kf = KFold(n_splits=CONFIG['k_folds'], shuffle=True, random_state=42)
    rmse_scores = []
    
    for train_idx, val_idx in kf.split(X_train):
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]
        
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
        
        model.fit(X_tr, y_tr, epochs=params['epochs'], batch_size=params['batch_size'],
                  validation_split=0.2, verbose=0, callbacks=[early_stopping, reduce_lr])
        y_pred = model.predict(X_val, verbose=0).flatten()
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        rmse_scores.append(rmse)
    
    return np.mean(rmse_scores)

# Hàm chạy PSO
def run_pso_lstm(X_train, y_train, num_particles=15, max_iter=10):
    dim = len(param_bounds)
    bounds_array = np.array(list(param_bounds.values()))
    
    particles = [random_particle() for _ in range(num_particles)]
    velocities = [np.zeros(dim) for _ in range(num_particles)]
    p_best_positions = particles.copy()
    p_best_scores = [fitness_function(p, X_train, y_train) for p in particles]
    
    g_best_index = np.argmin(p_best_scores)
    g_best_position = p_best_positions[g_best_index]
    g_best_score = p_best_scores[g_best_index]
    
    w, c1, c2 = 0.5, 1.5, 1.5
    
    for iter in range(max_iter):
        print(f"\n🔁 Iteration {iter + 1}/{max_iter}")
        for i in range(num_particles):
            r1 = np.random.rand(dim)
            r2 = np.random.rand(dim)
            
            velocities[i] = (
                w * velocities[i]
                + c1 * r1 * (p_best_positions[i] - particles[i])
                + c2 * r2 * (g_best_position - particles[i])
            )
            
            particles[i] += velocities[i]
            particles[i] = np.clip(particles[i], bounds_array[:, 0], bounds_array[:, 1])
            particles[i][2] = max(particles[i][2], param_bounds['l2_reg'][0])
            
            score = fitness_function(particles[i], X_train, y_train)
            
            if score < p_best_scores[i]:
                p_best_scores[i] = score
                p_best_positions[i] = particles[i]
                
            if score < g_best_score:
                g_best_score = score
                g_best_position = particles[i]
                print(f"✅ Cập nhật g_best: Score = {g_best_score:.4f}")
    
    return g_best_position, g_best_score

# Hàm huấn luyện và đánh giá mô hình tối ưu
def train_and_evaluate(X_train, X_test, y_train, y_test, best_params):
    model = build_lstm_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        **{k: v for k, v in best_params.items() if k not in ['batch_size', 'epochs']}
    )
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
    
    kf = KFold(n_splits=CONFIG['k_folds'], shuffle=True, random_state=42)
    metrics_all = []
    history_all = {'loss': [], 'val_loss': []}
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
        print(f"\n📂 Fold {fold + 1}/{CONFIG['k_folds']}")
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]
        
        history = model.fit(
            X_tr, y_tr, epochs=best_params['epochs'], batch_size=best_params['batch_size'],
            validation_split=0.2, verbose=0, callbacks=[early_stopping, reduce_lr]
        )
        y_pred = model.predict(X_val, verbose=0).flatten()
        metrics = calculate_metrics(y_val, y_pred)
        metrics_all.append(metrics)
        print(f"✅ Fold {fold + 1} RMSE: {metrics['rmse']:.4f}")
        
        history_all['loss'].append(history.history['loss'])
        history_all['val_loss'].append(history.history['val_loss'])
    
    # Đánh giá trên tập test
    y_pred_test = model.predict(X_test, verbose=0).flatten()
    test_metrics = calculate_metrics(y_test, y_pred_test)
    
    # Tính trung bình lịch sử huấn luyện
    max_len = max(len(h) for h in history_all['loss'])
    loss_avg = np.mean([np.pad(h, (0, max_len - len(h)), 'constant', constant_values=np.nan) for h in history_all['loss']], axis=0)
    val_loss_avg = np.mean([np.pad(h, (0, max_len - len(h)), 'constant', constant_values=np.nan) for h in history_all['val_loss']], axis=0)
    
    return model, test_metrics, metrics_all, loss_avg, val_loss_avg, y_pred_test

# Hàm trực quan hóa kết quả
def visualize_results(dataset_name, y_test, y_pred, loss_avg, val_loss_avg):
    if not CONFIG['visualize_results']:
        return
    
    plt.figure(figsize=(15, 12))
    
    # Loss trung bình
    plt.subplot(2, 2, 1)
    plt.plot(loss_avg, label='Training Loss')
    plt.plot(val_loss_avg, label='Validation Loss')
    plt.title('Average Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Huber Loss')
    plt.legend()
    
    # Predicted vs Actual
    plt.subplot(2, 2, 2)
    plt.scatter(y_test, y_pred, alpha=0.5)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    plt.title('Predicted vs Actual Effort')
    plt.xlabel('Actual Effort')
    plt.ylabel('Predicted Effort')
    
    # Error Distribution
    errors = y_test - y_pred
    plt.subplot(2, 2, 3)
    sns.histplot(errors, kde=True)
    plt.title('Error Distribution')
    plt.xlabel('Prediction Error')
    plt.ylabel('Frequency')
    
    plt.tight_layout()
    os.makedirs('visualizations', exist_ok=True)
    plt.savefig(f'visualizations/{dataset_name}_lstm_results.png')
    plt.close()

# Hàm chính để chạy pipeline
def run_pipeline(dataset_config):
    dataset_name = dataset_config['name']
    print(f"\n🚀 Chạy pipeline cho dataset: {dataset_name}")
    
    # Đọc và tiền xử lý dữ liệu
    X, y, df = load_and_preprocess_data(
        dataset_config['path'],
        dataset_config['feature_columns'],
        dataset_config['target_column'],
        dataset_config['numeric_columns']
    )
    
    # Tăng cường dữ liệu
    X_aug, y_aug = augment_data(X, y, CONFIG['noise_factor'], CONFIG['num_noise_copies'])
    
    # Reshape dữ liệu cho LSTM
    X_aug = reshape_for_lstm(X_aug, CONFIG['timesteps'])
    y_aug = y_aug[:X_aug.shape[0]]
    
    # Chia tập train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X_aug, y_aug, test_size=CONFIG['test_size'], random_state=42
    )
    
    # Chạy PSO
    print("🔍 Tìm siêu tham số tối ưu bằng PSO...")
    best_particle, best_score = run_pso_lstm(X_train, y_train, num_particles=15, max_iter=10)
    best_params = decode_particle(best_particle)
    print(f"🏆 Siêu tham số tốt nhất: {best_params}")
    print(f"📉 Score tốt nhất: {best_score:.4f}")
    
    # Huấn luyện và đánh giá
    model, test_metrics, metrics_all, loss_avg, val_loss_avg, y_pred_test = train_and_evaluate(
        X_train, X_test, y_train, y_test, best_params
    )
    
    # In kết quả
    print("\n📈 Kết quả đánh giá trên tập test:")
    for metric, value in test_metrics.items():
        print(f"📌 {metric.upper():<7}: {value:.4f}" if not np.isnan(value) else f"📌 {metric.upper():<7}: NaN")
    
    # Lưu kết quả
    if CONFIG['save_results']:
        results_df = pd.DataFrame([test_metrics])
        os.makedirs('results', exist_ok=True)
        results_df.to_csv(f'results/{dataset_name}_lstm_results.csv', index=False)
        print(f"\nĐã lưu kết quả vào 'results/{dataset_name}_lstm_results.csv'")
    
    # Lưu mô hình
    if CONFIG['save_model']:
        os.makedirs('models', exist_ok=True)
        model.save(f'models/{dataset_name}_lstm_model.keras')
        print(f"Đã lưu mô hình vào 'models/{dataset_name}_lstm_model.keras'")
    
    # Trực quan hóa
    visualize_results(dataset_name, y_test, y_pred_test, loss_avg, val_loss_avg)
    print(f"Đã lưu hình ảnh trực quan hóa vào 'visualizations/{dataset_name}_lstm_results.png'")

# Cấu hình các dataset
datasets = [
    {
        'name': 'desharnais',
        'path': 'desharnais1.1_processed_corrected.csv',
        'feature_columns': [
            'TeamExp', 'ManagerExp', 'YearEnd', 'Length', 'Transactions', 'Entities',
            'Adjustment', 'PointsAjust', 'StartYear', 'ProjectDurationYears',
            'Transactions_Entities', 'Effort_PointsAjust', 'Effort_per_PointsAjust',
            'Transactions_per_Entities', 'Language_b\'1\'', 'Language_b\'2\'',
            'Language_b\'3\'', 'HighEffort', 'HighPointsAjust'
        ],
        'target_column': 'Effort',
        'numeric_columns': [
            'TeamExp', 'ManagerExp', 'YearEnd', 'Length', 'Transactions', 'Entities',
            'Adjustment', 'PointsAjust', 'StartYear', 'ProjectDurationYears',
            'Transactions_Entities', 'Effort_PointsAjust', 'Effort_per_PointsAjust',
            'Transactions_per_Entities', 'Language_b\'1\'', 'Language_b\'2\'',
            'Language_b\'3\'', 'HighEffort', 'HighPointsAjust'
        ]
    }
]

# Chạy pipeline cho từng dataset
if __name__ == "__main__":
    for dataset in datasets:
        run_pipeline(dataset)


🚀 Chạy pipeline cho dataset: desharnais
🔍 Tìm siêu tham số tối ưu bằng PSO...

🔁 Iteration 1/10
✅ Cập nhật g_best: Score = 0.2331
✅ Cập nhật g_best: Score = 0.2070

🔁 Iteration 2/10
✅ Cập nhật g_best: Score = 0.2040
✅ Cập nhật g_best: Score = 0.1954
✅ Cập nhật g_best: Score = 0.1946

🔁 Iteration 3/10
✅ Cập nhật g_best: Score = 0.1709

🔁 Iteration 4/10

🔁 Iteration 5/10

🔁 Iteration 6/10

🔁 Iteration 7/10

🔁 Iteration 8/10

🔁 Iteration 9/10

🔁 Iteration 10/10
🏆 Siêu tham số tốt nhất: {'lstm_units': 17, 'dense_units': 10, 'l2_reg': np.float64(0.05290511907499294), 'dropout_rate': np.float64(0.2388252925394604), 'learning_rate': np.float64(0.01028603584658312), 'batch_size': 13, 'epochs': 105}
📉 Score tốt nhất: 0.1709

📂 Fold 1/3
✅ Fold 1 RMSE: 0.1741

📂 Fold 2/3
✅ Fold 2 RMSE: 0.1602

📂 Fold 3/3
✅ Fold 3 RMSE: 0.1102

📈 Kết quả đánh giá trên tập test:
📌 MAE    : 0.1038
📌 MSE    : 0.0185
📌 RMSE   : 0.1361
📌 R2     : 0.9869
📌 MMRE   : 3.4108
📌 MDMRE  : 0.0612
📌 PRED25 : 88.2353

Đã lưu kế

# RBFN

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.cluster import KMeans
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import os

# Thiết lập seed để tái lập
np.random.seed(42)
tf.random.set_seed(42)

# Cấu hình bật/tắt các bước
CONFIG = {
    'apply_gaussian_noise': True,  # Bật/tắt tăng cường dữ liệu bằng nhiễu Gaussian
    'normalize_features': False,   # Bật/tắt chuẩn hóa đặc trưng
    'log_transform_target': False, # Bật/tắt biến đổi log cho biến mục tiêu
    'save_model': True,            # Bật/tắt lưu mô hình
    'save_results': True,          # Bật/tắt lưu kết quả đánh giá
    'visualize_results': True,     # Bật/tắt trực quan hóa
    'noise_factor': 0.01,          # Độ lớn nhiễu Gaussian
    'num_noise_copies': 2,         # Số bản sao nhiễu
    'test_size': 0.15,             # Tỷ lệ tập test
    'k_folds': 3,                  # Số fold trong cross-validation
}

# Hàm tính các chỉ số đánh giá
def calculate_metrics(y_true, y_pred):
    mask = y_true > 0
    if np.sum(mask) == 0:
        return {
            'mae': np.nan, 'mse': np.nan, 'rmse': np.nan, 'r2': np.nan,
            'mmre': np.nan, 'mdmre': np.nan, 'pred25': np.nan
        }
    
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    mre = np.abs(y_true[mask] - y_pred[mask]) / y_true[mask]
    mmre = np.mean(mre)
    mdmre = np.median(mre)
    pred25 = np.mean(mre <= 0.25) * 100
    
    return {
        'mae': mae, 'mse': mse, 'rmse': rmse, 'r2': r2,
        'mmre': mmre, 'mdmre': mdmre, 'pred25': pred25
    }

# Hàm đọc và tiền xử lý dữ liệu
def load_and_preprocess_data(dataset_path, feature_columns, target_column, numeric_columns):
    df = pd.read_csv(dataset_path)
    
    # Kiểm tra cột tồn tại
    missing_cols = [col for col in feature_columns + [target_column] if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing columns in dataset: {missing_cols}")
    
    # Kiểm tra kiểu dữ liệu
    for col in numeric_columns:
        if not np.issubdtype(df[col].dtype, np.number):
            raise ValueError(f"Column {col} is not numeric")
    if not np.issubdtype(df[target_column].dtype, np.number):
        raise ValueError(f"Target column {target_column} is not numeric")
    
    # Kiểm tra NaN
    X = df[feature_columns].values
    y = df[target_column].values
    if np.any(np.isnan(X)) or np.any(np.isnan(y)):
        raise ValueError("Data contains NaN values")
    
    # Chuẩn hóa đặc trưng nếu bật
    if CONFIG['normalize_features']:
        X = (X - X.mean(axis=0)) / (X.std(axis=0) + 1e-8)
    
    # Biến đổi log cho mục tiêu nếu bật
    if CONFIG['log_transform_target']:
        y = np.log1p(y)
    
    return X, y, df

# Hàm tăng cường dữ liệu bằng nhiễu Gaussian
def augment_data(X, y, noise_factor=0.01, num_copies=2):
    if not CONFIG['apply_gaussian_noise']:
        return X, y
    
    X_aug = X.copy()
    y_aug = y.copy()
    
    for _ in range(num_copies):
        noise = np.random.normal(loc=0, scale=noise_factor, size=X.shape)
        X_noisy = X + noise
        X_aug = np.vstack((X_aug, X_noisy))
        y_aug = np.hstack((y_aug, y))
    
    return X_aug, y_aug

# Tầng RBF tùy chỉnh
class RBFLayer(tf.keras.layers.Layer):
    def __init__(self, units, sigma, centers, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.units = units
        self.sigma = float(sigma)
        self.centers = tf.Variable(centers, trainable=False, dtype=tf.float32, name='centers')
    
    def build(self, input_shape):
        self.rbf_weights = self.add_weight(
            name='rbf_weights',
            shape=(self.units, 1),
            initializer='glorot_uniform',
            trainable=True
        )
        self.rbf_biases = self.add_weight(
            name='rbf_biases',
            shape=(1,),
            initializer='zeros',
            trainable=True
        )
        super(RBFLayer, self).build(input_shape)
    
    def call(self, inputs):
        diff = tf.expand_dims(inputs, axis=1) - tf.expand_dims(self.centers, axis=0)
        l2 = tf.reduce_sum(tf.square(diff), axis=-1)
        output = tf.exp(-l2 / (2.0 * self.sigma ** 2))
        return tf.matmul(output, self.rbf_weights) + self.rbf_biases
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], 1)
    
    def get_config(self):
        config = super(RBFLayer, self).get_config()
        config.update({
            'units': self.units,
            'sigma': float(self.sigma),
            'centers': self.centers.numpy().tolist()
        })
        return config

# Hàm xây dựng mô hình RBFN
def build_rbfn_model(input_dim, num_centers=10, sigma=1.0, learning_rate=0.001, X_train=None):
    if X_train is None:
        raise ValueError("X_train must be provided to initialize centers with KMeans")
    
    # Khởi tạo centers bằng KMeans
    n_clusters = max(int(num_centers), 1)
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(X_train)
    centers = np.array(kmeans.cluster_centers_, dtype=np.float32)
    
    model = Sequential([
        Input(shape=(input_dim,)),
        RBFLayer(units=int(num_centers), sigma=sigma, centers=centers, name='rbf_layer'),
        Dense(1, activation='linear', name='output_layer')
    ])
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=tf.keras.losses.Huber(), metrics=['mae'])
    return model

# Không gian siêu tham số cho RBFN
param_bounds = {
    'num_centers': (5, 20),
    'sigma': (0.1, 2.0),
    'learning_rate': (1e-4, 1e-2),
    'batch_size': (16, 32),
    'epochs': (50, 100)
}

# Hàm mã hóa & giải mã particle
def random_particle():
    return np.array([
        np.random.randint(param_bounds['num_centers'][0], param_bounds['num_centers'][1] + 1),
        np.random.uniform(param_bounds['sigma'][0], param_bounds['sigma'][1]),
        np.random.uniform(param_bounds['learning_rate'][0], param_bounds['learning_rate'][1]),
        np.random.randint(param_bounds['batch_size'][0], param_bounds['batch_size'][1] + 1),
        np.random.randint(param_bounds['epochs'][0], param_bounds['epochs'][1] + 1)
    ])

def decode_particle(particle):
    params = {
        'num_centers': int(particle[0]),
        'sigma': float(particle[1]),
        'learning_rate': float(particle[2]),
        'batch_size': int(particle[3]),
        'epochs': int(particle[4])
    }
    return params

# Hàm fitness cho PSO
def fitness_function(particle, X_train, y_train):
    params = decode_particle(particle)
    model = build_rbfn_model(
        input_dim=X_train.shape[1],
        X_train=X_train,
        **{k: v for k, v in params.items() if k not in ['batch_size', 'epochs']}
    )
    
    kf = KFold(n_splits=CONFIG['k_folds'], shuffle=True, random_state=42)
    rmse_scores = []
    
    for train_idx, val_idx in kf.split(X_train):
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]
        
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
        
        model.fit(X_tr, y_tr, epochs=params['epochs'], batch_size=params['batch_size'],
                  validation_split=0.2, verbose=0, callbacks=[early_stopping, reduce_lr])
        y_pred = model.predict(X_val, verbose=0).flatten()
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        rmse_scores.append(rmse)
    
    return np.mean(rmse_scores)

# Hàm chạy PSO
def run_pso_rbfn(X_train, y_train, num_particles=15, max_iter=10):
    dim = len(param_bounds)
    bounds_array = np.array(list(param_bounds.values()))
    
    particles = [random_particle() for _ in range(num_particles)]
    velocities = [np.zeros(dim) for _ in range(num_particles)]
    p_best_positions = particles.copy()
    p_best_scores = [fitness_function(p, X_train, y_train) for p in particles]
    
    g_best_index = np.argmin(p_best_scores)
    g_best_position = p_best_positions[g_best_index]
    g_best_score = p_best_scores[g_best_index]
    
    w, c1, c2 = 0.5, 1.5, 1.5
    
    for iter in range(max_iter):
        print(f"\n🔁 Iteration {iter + 1}/{max_iter}")
        for i in range(num_particles):
            r1 = np.random.rand(dim)
            r2 = np.random.rand(dim)
            
            velocities[i] = (
                w * velocities[i]
                + c1 * r1 * (p_best_positions[i] - particles[i])
                + c2 * r2 * (g_best_position - particles[i])
            )
            
            particles[i] += velocities[i]
            particles[i] = np.clip(particles[i], bounds_array[:, 0], bounds_array[:, 1])
            particles[i][1] = max(particles[i][1], param_bounds['sigma'][0])
            
            score = fitness_function(particles[i], X_train, y_train)
            
            if score < p_best_scores[i]:
                p_best_scores[i] = score
                p_best_positions[i] = particles[i]
                
            if score < g_best_score:
                g_best_score = score
                g_best_position = particles[i]
                print(f"✅ Cập nhật g_best: Score = {g_best_score:.4f}")
    
    return g_best_position, g_best_score

# Hàm huấn luyện và đánh giá mô hình tối ưu
def train_and_evaluate(X_train, X_test, y_train, y_test, best_params):
    model = build_rbfn_model(
        input_dim=X_train.shape[1],
        X_train=X_train,
        **{k: v for k, v in best_params.items() if k not in ['batch_size', 'epochs']}
    )
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
    
    kf = KFold(n_splits=CONFIG['k_folds'], shuffle=True, random_state=42)
    metrics_all = []
    history_all = {'loss': [], 'val_loss': []}
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
        print(f"\n📂 Fold {fold + 1}/{CONFIG['k_folds']}")
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]
        
        history = model.fit(
            X_tr, y_tr, epochs=best_params['epochs'], batch_size=best_params['batch_size'],
            validation_split=0.2, verbose=0, callbacks=[early_stopping, reduce_lr]
        )
        y_pred = model.predict(X_val, verbose=0).flatten()
        metrics = calculate_metrics(y_val, y_pred)
        metrics_all.append(metrics)
        print(f"✅ Fold {fold + 1} RMSE: {metrics['rmse']:.4f}")
        
        history_all['loss'].append(history.history['loss'])
        history_all['val_loss'].append(history.history['val_loss'])
    
    # Đánh giá trên tập test
    y_pred_test = model.predict(X_test, verbose=0).flatten()
    test_metrics = calculate_metrics(y_test, y_pred_test)
    
    # Tính trung bình lịch sử huấn luyện
    max_len = max(len(h) for h in history_all['loss'])
    loss_avg = np.mean([np.pad(h, (0, max_len - len(h)), 'constant', constant_values=np.nan) for h in history_all['loss']], axis=0)
    val_loss_avg = np.mean([np.pad(h, (0, max_len - len(h)), 'constant', constant_values=np.nan) for h in history_all['val_loss']], axis=0)
    
    return model, test_metrics, metrics_all, loss_avg, val_loss_avg, y_pred_test

# Hàm trực quan hóa kết quả
def visualize_results(dataset_name, y_test, y_pred, loss_avg, val_loss_avg):
    if not CONFIG['visualize_results']:
        return
    
    plt.figure(figsize=(15, 12))
    
    # Loss trung bình
    plt.subplot(2, 2, 1)
    plt.plot(loss_avg, label='Training Loss')
    plt.plot(val_loss_avg, label='Validation Loss')
    plt.title('Average Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Huber Loss')
    plt.legend()
    
    # Predicted vs Actual
    plt.subplot(2, 2, 2)
    plt.scatter(y_test, y_pred, alpha=0.5)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    plt.title('Predicted vs Actual Effort')
    plt.xlabel('Actual Effort')
    plt.ylabel('Predicted Effort')
    
    # Error Distribution
    errors = y_test - y_pred
    plt.subplot(2, 2, 3)
    sns.histplot(errors, kde=True)
    plt.title('Error Distribution')
    plt.xlabel('Prediction Error')
    plt.ylabel('Frequency')
    
    plt.tight_layout()
    os.makedirs('visualizations', exist_ok=True)
    plt.savefig(f'visualizations/{dataset_name}_rbfn_results.png')
    plt.close()

# Hàm chính để chạy pipeline
def run_pipeline(dataset_config):
    dataset_name = dataset_config['name']
    print(f"\n🚀 Chạy pipeline cho dataset: {dataset_name}")
    
    # Đọc và tiền xử lý dữ liệu
    X, y, df = load_and_preprocess_data(
        dataset_config['path'],
        dataset_config['feature_columns'],
        dataset_config['target_column'],
        dataset_config['numeric_columns']
    )
    
    # Tăng cường dữ liệu
    X_aug, y_aug = augment_data(X, y, CONFIG['noise_factor'], CONFIG['num_noise_copies'])
    
    # Chia tập train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X_aug, y_aug, test_size=CONFIG['test_size'], random_state=42
    )
    
    # Chạy PSO
    print("🔍 Tìm siêu tham số tối ưu bằng PSO...")
    best_particle, best_score = run_pso_rbfn(X_train, y_train, num_particles=15, max_iter=10)
    best_params = decode_particle(best_particle)
    print(f"🏆 Siêu tham số tốt nhất: {best_params}")
    print(f"📉 Score tốt nhất: {best_score:.4f}")
    
    # Huấn luyện và đánh giá
    model, test_metrics, metrics_all, loss_avg, val_loss_avg, y_pred_test = train_and_evaluate(
        X_train, X_test, y_train, y_test, best_params
    )
    
    # In kết quả
    print("\n📈 Kết quả đánh giá trên tập test:")
    for metric, value in test_metrics.items():
        print(f"📌 {metric.upper():<7}: {value:.4f}" if not np.isnan(value) else f"📌 {metric.upper():<7}: NaN")
    
    # Lưu kết quả
    if CONFIG['save_results']:
        results_df = pd.DataFrame([test_metrics])
        os.makedirs('results', exist_ok=True)
        results_df.to_csv(f'results/{dataset_name}_rbfn_results.csv', index=False)
        print(f"\nĐã lưu kết quả vào 'results/{dataset_name}_rbfn_results.csv'")
    
    # Lưu mô hình
    if CONFIG['save_model']:
        os.makedirs('models', exist_ok=True)
        model.save(f'models/{dataset_name}_rbfn_model.keras')
        print(f"Đã lưu mô hình vào 'models/{dataset_name}_rbfn_model.keras'")
    
    # Trực quan hóa
    visualize_results(dataset_name, y_test, y_pred_test, loss_avg, val_loss_avg)
    print(f"Đã lưu hình ảnh trực quan hóa vào 'visualizations/{dataset_name}_rbfn_results.png'")

# Cấu hình các dataset
datasets = [
    {
        'name': 'desharnais',
        'path': 'desharnais1.1_processed_corrected.csv',
        'feature_columns': [
            'TeamExp', 'ManagerExp', 'YearEnd', 'Length', 'Transactions', 'Entities',
            'Adjustment', 'PointsAjust', 'StartYear', 'ProjectDurationYears',
            'Transactions_Entities', 'Effort_PointsAjust', 'Effort_per_PointsAjust',
            'Transactions_per_Entities', 'Language_b\'1\'', 'Language_b\'2\'',
            'Language_b\'3\'', 'HighEffort', 'HighPointsAjust'
        ],
        'target_column': 'Effort',
        'numeric_columns': [
            'TeamExp', 'ManagerExp', 'YearEnd', 'Length', 'Transactions', 'Entities',
            'Adjustment', 'PointsAjust', 'StartYear', 'ProjectDurationYears',
            'Transactions_Entities', 'Effort_PointsAjust', 'Effort_per_PointsAjust',
            'Transactions_per_Entities', 'Language_b\'1\'', 'Language_b\'2\'',
            'Language_b\'3\'', 'HighEffort', 'HighPointsAjust'
        ]
    }
]

# Chạy pipeline cho từng dataset
if __name__ == "__main__":
    for dataset in datasets:
        run_pipeline(dataset)


🚀 Chạy pipeline cho dataset: desharnais
🔍 Tìm siêu tham số tối ưu bằng PSO...

🔁 Iteration 1/10
✅ Cập nhật g_best: Score = 0.4931
✅ Cập nhật g_best: Score = 0.4696
✅ Cập nhật g_best: Score = 0.4233

🔁 Iteration 2/10
✅ Cập nhật g_best: Score = 0.3826
✅ Cập nhật g_best: Score = 0.3691
✅ Cập nhật g_best: Score = 0.3667

🔁 Iteration 3/10
✅ Cập nhật g_best: Score = 0.3663
✅ Cập nhật g_best: Score = 0.3646
✅ Cập nhật g_best: Score = 0.3622

🔁 Iteration 4/10
✅ Cập nhật g_best: Score = 0.3620

🔁 Iteration 5/10

🔁 Iteration 6/10

🔁 Iteration 7/10
✅ Cập nhật g_best: Score = 0.3596

🔁 Iteration 8/10

🔁 Iteration 9/10

🔁 Iteration 10/10
🏆 Siêu tham số tốt nhất: {'num_centers': 22, 'sigma': 2.14376277835498, 'learning_rate': 0.008146268376391983, 'batch_size': 11, 'epochs': 43}
📉 Score tốt nhất: 0.3596

📂 Fold 1/3
✅ Fold 1 RMSE: 0.3454

📂 Fold 2/3
✅ Fold 2 RMSE: 0.3723

📂 Fold 3/3
✅ Fold 3 RMSE: 0.3257

📈 Kết quả đánh giá trên tập test:
📌 MAE    : 0.3087
📌 MSE    : 0.2034
📌 RMSE   : 0.4510
📌 R2   

# RNN

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import os

# Thiết lập seed để tái lập
np.random.seed(42)
tf.random.set_seed(42)

# Cấu hình bật/tắt các bước
CONFIG = {
    'apply_gaussian_noise': True,  # Bật/tắt tăng cường dữ liệu bằng nhiễu Gaussian
    'normalize_features': False,   # Bật/tắt chuẩn hóa đặc trưng
    'log_transform_target': False, # Bật/tắt biến đổi log cho biến mục tiêu
    'save_model': True,            # Bật/tắt lưu mô hình
    'save_results': True,          # Bật/tắt lưu kết quả đánh giá
    'visualize_results': True,     # Bật/tắt trực quan hóa
    'noise_factor': 0.01,          # Độ lớn nhiễu Gaussian
    'num_noise_copies': 2,         # Số bản sao nhiễu
    'test_size': 0.15,             # Tỷ lệ tập test
    'k_folds': 3,                  # Số fold trong cross-validation
    'timesteps': 1                 # Số bước thời gian cho RNN 
}

# Hàm tính các chỉ số đánh giá
def calculate_metrics(y_true, y_pred):
    mask = y_true > 0
    if np.sum(mask) == 0:
        return {
            'mae': np.nan, 'mse': np.nan, 'rmse': np.nan, 'r2': np.nan,
            'mmre': np.nan, 'mdmre': np.nan, 'pred25': np.nan
        }
    
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    mre = np.abs(y_true[mask] - y_pred[mask]) / y_true[mask]
    mmre = np.mean(mre)
    mdmre = np.median(mre)
    pred25 = np.mean(mre <= 0.25) * 100
    
    return {
        'mae': mae, 'mse': mse, 'rmse': rmse, 'r2': r2,
        'mmre': mmre, 'mdmre': mdmre, 'pred25': pred25
    }

# Hàm đọc và tiền xử lý dữ liệu
def load_and_preprocess_data(dataset_path, feature_columns, target_column, numeric_columns):
    df = pd.read_csv(dataset_path)
    
    # Kiểm tra cột tồn tại
    missing_cols = [col for col in feature_columns + [target_column] if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing columns in dataset: {missing_cols}")
    
    # Kiểm tra kiểu dữ liệu
    for col in numeric_columns:
        if not np.issubdtype(df[col].dtype, np.number):
            raise ValueError(f"Column {col} is not numeric")
    if not np.issubdtype(df[target_column].dtype, np.number):
        raise ValueError(f"Target column {target_column} is not numeric")
    
    # Kiểm tra NaN
    X = df[feature_columns].values
    y = df[target_column].values
    if np.any(np.isnan(X)) or np.any(np.isnan(y)):
        raise ValueError("Data contains NaN values")
    
    # Chuẩn hóa đặc trưng nếu bật
    if CONFIG['normalize_features']:
        X = (X - X.mean(axis=0)) / (X.std(axis=0) + 1e-8)
    
    # Biến đổi log cho mục tiêu nếu bật
    if CONFIG['log_transform_target']:
        y = np.log1p(y)
    
    return X, y, df

# Hàm tăng cường dữ liệu bằng nhiễu Gaussian
def augment_data(X, y, noise_factor=0.01, num_copies=2):
    if not CONFIG['apply_gaussian_noise']:
        return X, y
    
    X_aug = X.copy()
    y_aug = y.copy()
    
    for _ in range(num_copies):
        noise = np.random.normal(loc=0, scale=noise_factor, size=X.shape)
        X_noisy = X + noise
        X_aug = np.vstack((X_aug, X_noisy))
        y_aug = np.hstack((y_aug, y))
    
    return X_aug, y_aug

# Hàm reshape dữ liệu cho RNN
def reshape_for_rnn(X, timesteps=1):
    # Reshape dữ liệu thành [samples, timesteps, features]
    n_samples, n_features = X.shape
    n_timesteps = timesteps
    n_new_samples = n_samples // n_timesteps
    X_reshaped = X[:n_new_samples * n_timesteps].reshape(n_new_samples, n_timesteps, n_features)
    return X_reshaped

# Hàm xây dựng mô hình RNN
def build_rnn_model(input_shape, rnn_units=16, dense_units=8, l2_reg=0.01, dropout_rate=0.3, learning_rate=0.001):
    l2_reg = max(l2_reg, 0.001)
    model = Sequential([
        Input(shape=input_shape),
        SimpleRNN(rnn_units, activation='tanh', return_sequences=False, kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        Dropout(dropout_rate),
        Dense(dense_units, activation='relu', kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        Dropout(dropout_rate),
        Dense(1, activation='linear')
    ])
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=tf.keras.losses.Huber(), metrics=['mae'])
    return model

# Không gian siêu tham số cho RNN
param_bounds = {
    'rnn_units': (8, 32),
    'dense_units': (4, 16),
    'l2_reg': (0.001, 0.05),
    'dropout_rate': (0.2, 0.4),
    'learning_rate': (1e-4, 1e-2),
    'batch_size': (16, 32),
    'epochs': (50, 100)
}

# Hàm mã hóa & giải mã particle
def random_particle():
    return np.array([
        np.random.randint(param_bounds['rnn_units'][0], param_bounds['rnn_units'][1] + 1),
        np.random.randint(param_bounds['dense_units'][0], param_bounds['dense_units'][1] + 1),
        np.random.uniform(param_bounds['l2_reg'][0], param_bounds['l2_reg'][1]),
        np.random.uniform(param_bounds['dropout_rate'][0], param_bounds['dropout_rate'][1]),
        np.random.uniform(param_bounds['learning_rate'][0], param_bounds['learning_rate'][1]),
        np.random.randint(param_bounds['batch_size'][0], param_bounds['batch_size'][1] + 1),
        np.random.randint(param_bounds['epochs'][0], param_bounds['epochs'][1] + 1)
    ])

def decode_particle(particle):
    params = {
        'rnn_units': int(particle[0]),
        'dense_units': int(particle[1]),
        'l2_reg': particle[2],
        'dropout_rate': particle[3],
        'learning_rate': particle[4],
        'batch_size': int(particle[5]),
        'epochs': int(particle[6])
    }
    params['l2_reg'] = max(params['l2_reg'], 0.001)
    return params

# Hàm fitness cho PSO
def fitness_function(particle, X_train, y_train):
    params = decode_particle(particle)
    model = build_rnn_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        **{k: v for k, v in params.items() if k not in ['batch_size', 'epochs']}
    )
    
    kf = KFold(n_splits=CONFIG['k_folds'], shuffle=True, random_state=42)
    rmse_scores = []
    
    for train_idx, val_idx in kf.split(X_train):
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]
        
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
        
        model.fit(X_tr, y_tr, epochs=params['epochs'], batch_size=params['batch_size'],
                  validation_split=0.2, verbose=0, callbacks=[early_stopping, reduce_lr])
        y_pred = model.predict(X_val, verbose=0).flatten()
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        rmse_scores.append(rmse)
    
    return np.mean(rmse_scores)

# Hàm chạy PSO
def run_pso_rnn(X_train, y_train, num_particles=15, max_iter=10):
    dim = len(param_bounds)
    bounds_array = np.array(list(param_bounds.values()))
    
    particles = [random_particle() for _ in range(num_particles)]
    velocities = [np.zeros(dim) for _ in range(num_particles)]
    p_best_positions = particles.copy()
    p_best_scores = [fitness_function(p, X_train, y_train) for p in particles]
    
    g_best_index = np.argmin(p_best_scores)
    g_best_position = p_best_positions[g_best_index]
    g_best_score = p_best_scores[g_best_index]
    
    w, c1, c2 = 0.5, 1.5, 1.5
    
    for iter in range(max_iter):
        print(f"\n🔁 Iteration {iter + 1}/{max_iter}")
        for i in range(num_particles):
            r1 = np.random.rand() * np.ones(dim)
            r2 = np.random.uniform(0, 1, dim)
            
            velocities[i] = (
                w * velocities[i]
                + c1 * r1 * (p_best_positions[i] - particles[i])
                + c2 * r2 * (g_best_position - particles[i])
            )
            
            particles[i] += velocities[i]
            particles[i] = np.clip(particles[i], bounds_array[:, 0], bounds_array[:, -1])
            particles[i][2] = max(particles[i][2], param_bounds['l2_reg'][0])
            
            score = fitness_function(particles[i], X_train, y_train)
            
            if score < p_best_scores[i]:
                p_best_scores[i] = score
                p_best_positions[i] = particles[i]
                
            if score < g_best_score:
                g_best_score = score
                g_best_position = particles[i]
                print(f"✅ Cập nhật g_best: Score = {g_best_score:.4f}")
                
    return g_best_position, g_best_score

# Hàm huấn luyện và đánh giá mô hình tối ưu
def train_and_evaluate(X_train, X_test, y_train, y_test, best_params):
    model = build_rnn_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        **{k: v for k, v in best_params.items() if k not in ['batch_size', 'epochs']}
    )
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
    
    kf = KFold(n_splits=CONFIG['k_folds'], shuffle=True, random_state=42)
    metrics_all = []
    history_all = {'loss': [], 'val_loss': []}
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
        print(f"\n📌 Fold {fold + 1}/{CONFIG['k_folds']}")
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]
        
        history = model.fit(
            X_tr, y_tr, epochs=best_params['epochs'], batch_size=best_params['batch_size'],
            validation_split=0.2, verbose=0, callbacks=[early_stopping, reduce_lr]
        )
        y_pred = model.predict(X_val, verbose=0).flatten()
        metrics = calculate_metrics(y_val, y_pred)
        metrics_all.append(metrics)
        print(f"✅ Fold {fold + 1} RMSE: {metrics['rmse']:.4f}")
        
        history_all['loss'].append(history.history['loss'])
        history_all['val_loss'].append(history.history['val_loss'])
    
    # Đánh giá trên tập test
    y_pred_test = model.predict(X_test, verbose=0).flatten()
    test_metrics = calculate_metrics(y_test, y_pred_test)
    
    # Tính trung bình lịch sử huấn luyện
    max_len = max(len(h) for h in history_all['loss'])
    loss_avg = np.mean([np.pad(h, (0, max_len - len(h)), 'constant', constant_values=np.nan) for h in history_all['loss']], axis=0)
    val_loss_avg = np.mean([np.pad(h, (0, max_len - len(h)), 'constant', constant_values=np.nan) for h in history_all['val_loss']], axis=0)
    
    return model, test_metrics, metrics_all, loss_avg, val_loss_avg, y_pred_test

# Hàm trực quan hóa kết quả
def visualize_results(dataset_name, y_test, y_pred, loss_avg, val_loss_avg):
    if not CONFIG['visualize_results']:
        return
    
    plt.figure(figsize=(15, 12))
    
    # Loss trung bình
    plt.subplot(2, 2, 1)
    plt.plot(loss_avg, label='Training Loss')
    plt.plot(val_loss_avg, label='Validation Loss')
    plt.title('Average Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Huber Loss')
    plt.legend()
    
    # Predicted vs Actual
    plt.subplot(2, 2, 2)
    plt.scatter(y_test, y_pred, alpha=0.5)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    plt.title('Predicted vs Actual Effort')
    plt.xlabel('Actual Effort')
    plt.ylabel('Predicted Effort')
    
    # Error Distribution
    errors = y_test - y_pred
    plt.subplot(2, 2, 3)
    sns.histplot(errors, kde=True)
    plt.title('Error Distribution')
    plt.xlabel('Prediction Error')
    plt.ylabel('Frequency')
    
    plt.tight_layout()
    os.makedirs('visualizations', exist_ok=True)
    plt.savefig(f'visualizations/{dataset_name}_rnn_results.png')
    plt.close()

# Hàm chính để chạy pipeline
def run_pipeline(dataset_config):
    dataset_name = dataset_config['name']
    print(f"\n🚀 Chạy pipeline cho dataset: {dataset_name}")
    
    # Đọc và tiền xử lý dữ liệu
    X, y, df = load_and_preprocess_data(
        dataset_config['path'],
        dataset_config['feature_columns'],
        dataset_config['target_column'],
        dataset_config['numeric_columns']
    )
    
    # Tăng cường dữ liệu
    X_aug, y_aug = augment_data(X, y, CONFIG['noise_factor'], CONFIG['num_noise_copies'])
    
    # Reshape dữ liệu cho RNN
    X_aug = reshape_for_rnn(X_aug, CONFIG['timesteps'])
    y_aug = y_aug[:X_aug.shape[0]]  # Cắt y để khớp số mẫu sau reshape
    
    # Chia tập train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X_aug, y_aug, test_size=CONFIG['test_size'], random_state=42
    )
    
    # Chạy PSO
    print("🔍 Tìm siêu tham số tối ưu bằng PSO...")
    best_particle, best_score = run_pso_rnn(X_train, y_train, num_particles=15, max_iter=10)
    best_params = decode_particle(best_particle)
    print(f"🏆 Siêu tham số tốt nhất: {best_params}")
    print(f"📉 Score tốt nhất: {best_score:.4f}")
    
    # Huấn luyện và đánh giá
    model, test_metrics, metrics_all, loss_avg, val_loss_avg, y_pred_test = train_and_evaluate(
        X_train, X_test, y_train, y_test, best_params
    )
    
    # In kết quả
    print("\n📈 Kết quả đánh giá trên tập test:")
    for metric, value in test_metrics.items():
        print(f"📌 {metric.upper():<7}: {value:.4f}" if not np.isnan(value) else f"📌 {metric.upper():<7}: NaN")
    
    # Lưu kết quả
    if CONFIG['save_results']:
        results_df = pd.DataFrame([test_metrics])
        os.makedirs('results', exist_ok=True)
        results_df.to_csv(f'results/{dataset_name}_rnn_results.csv', index=False)
        print(f"\nĐã lưu kết quả vào 'results/{dataset_name}_rnn_results.csv'")
    
    # Lưu mô hình
    if CONFIG['save_model']:
        os.makedirs('models', exist_ok=True)
        model.save(f'models/{dataset_name}_rnn_model.keras')
        print(f"Đã lưu mô hình vào 'models/{dataset_name}_rnn_model.keras'")
    
    # Trực quan hóa
    visualize_results(dataset_name, y_test, y_pred_test, loss_avg, val_loss_avg)
    print(f"Đã lưu hình ảnh trực quan hóa vào 'visualizations/{dataset_name}_rnn_results.png'")

# Cấu hình các dataset
datasets = [
    {
        'name': 'desharnais',
        'path': 'desharnais1.1_processed_corrected.csv',
        'feature_columns': [
            'TeamExp', 'ManagerExp', 'YearEnd', 'Length', 'Transactions', 'Entities',
            'Adjustment', 'PointsAjust', 'StartYear', 'ProjectDurationYears',
            'Transactions_Entities', 'Effort_PointsAjust', 'Effort_per_PointsAjust',
            'Transactions_per_Entities', 'Language_b\'1\'', 'Language_b\'2\'',
            'Language_b\'3\'', 'HighEffort', 'HighPointsAjust'
        ],
        'target_column': 'Effort',
        'numeric_columns': [
            'TeamExp', 'ManagerExp', 'YearEnd', 'Length', 'Transactions', 'Entities',
            'Adjustment', 'PointsAjust', 'StartYear', 'ProjectDurationYears',
            'Transactions_Entities', 'Effort_PointsAjust', 'Effort_per_PointsAjust',
            'Transactions_per_Entities', 'Language_b\'1\'', 'Language_b\'2\'',
            'Language_b\'3\'', 'HighEffort', 'HighPointsAjust'
        ]
    }
]

# Chạy pipeline cho từng dataset
if __name__ == "__main__":
    for dataset in datasets:
        run_pipeline(dataset)


🚀 Chạy pipeline cho dataset: desharnais
🔍 Tìm siêu tham số tối ưu bằng PSO...

🔁 Iteration 1/10

🔁 Iteration 2/10
✅ Cập nhật g_best: Score = 0.1618

🔁 Iteration 3/10
✅ Cập nhật g_best: Score = 0.1569
✅ Cập nhật g_best: Score = 0.1564

🔁 Iteration 4/10
✅ Cập nhật g_best: Score = 0.1502
✅ Cập nhật g_best: Score = 0.1476

🔁 Iteration 5/10
✅ Cập nhật g_best: Score = 0.1470

🔁 Iteration 6/10

🔁 Iteration 7/10

🔁 Iteration 8/10

🔁 Iteration 9/10

🔁 Iteration 10/10
🏆 Siêu tham số tốt nhất: {'rnn_units': 19, 'dense_units': 6, 'l2_reg': np.float64(0.04578883850996157), 'dropout_rate': np.float64(0.1755520424929638), 'learning_rate': np.float64(0.009449979979976523), 'batch_size': 13, 'epochs': 104}
📉 Score tốt nhất: 0.1470

📌 Fold 1/3
✅ Fold 1 RMSE: 0.1408

📌 Fold 2/3
✅ Fold 2 RMSE: 0.1700

📌 Fold 3/3
✅ Fold 3 RMSE: 0.1077

📈 Kết quả đánh giá trên tập test:
📌 MAE    : 0.0879
📌 MSE    : 0.0145
📌 RMSE   : 0.1205
📌 R2     : 0.9897
📌 MMRE   : 3.2531
📌 MDMRE  : 0.0423
📌 PRED25 : 88.2353

Đã lưu kết

: 

# BiLSTM

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import os

# Thiết lập seed để tái lập
np.random.seed(42)
tf.random.set_seed(42)

# Cấu hình bật/tắt các bước
CONFIG = {
    'apply_gaussian_noise': True,  # Bật/tắt tăng cường dữ liệu bằng nhiễu Gaussian
    'normalize_features': False,   # Bật/tắt chuẩn hóa đặc trưng
    'log_transform_target': False, # Bật/tắt biến đổi log cho biến mục tiêu
    'save_model': True,            # Bật/tắt lưu mô hình
    'save_results': True,          # Bật/tắt lưu kết quả đánh giá
    'visualize_results': True,     # Bật/tắt trực quan hóa
    'noise_factor': 0.01,          # Độ lớn nhiễu Gaussian
    'num_noise_copies': 2,         # Số bản sao nhiễu
    'test_size': 0.15,             # Tỷ lệ tập test
    'k_folds': 3,                  # Số fold trong cross-validation
    'timesteps': 1                 # Số bước thời gian cho BiLSTM 
}

# Hàm tính các chỉ số đánh giá
def calculate_metrics(y_true, y_pred):
    mask = y_true > 0
    if np.sum(mask) == 0:
        return {
            'mae': np.nan, 'mse': np.nan, 'rmse': np.nan, 'r2': np.nan,
            'mmre': np.nan, 'mdmre': np.nan, 'pred25': np.nan
        }
    
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    mre = np.abs(y_true[mask] - y_pred[mask]) / y_true[mask]
    mmre = np.mean(mre)
    mdmre = np.median(mre)
    pred25 = np.mean(mre <= 0.25) * 100
    
    return {
        'mae': mae, 'mse': mse, 'rmse': rmse, 'r2': r2,
        'mmre': mmre, 'mdmre': mdmre, 'pred25': pred25
    }

# Hàm đọc và tiền xử lý dữ liệu
def load_and_preprocess_data(dataset_path, feature_columns, target_column, numeric_columns):
    df = pd.read_csv(dataset_path)
    
    # Kiểm tra cột tồn tại
    missing_cols = [col for col in feature_columns + [target_column] if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing columns in dataset: {missing_cols}")
    
    # Kiểm tra kiểu dữ liệu
    for col in numeric_columns:
        if not np.issubdtype(df[col].dtype, np.number):
            raise ValueError(f"Column {col} is not numeric")
    if not np.issubdtype(df[target_column].dtype, np.number):
        raise ValueError(f"Target column {target_column} is not numeric")
    
    # Kiểm tra NaN
    X = df[feature_columns].values
    y = df[target_column].values
    if np.any(np.isnan(X)) or np.any(np.isnan(y)):
        raise ValueError("Data contains NaN values")
    
    # Chuẩn hóa đặc trưng nếu bật
    if CONFIG['normalize_features']:
        X = (X - X.mean(axis=0)) / (X.std(axis=0) + 1e-8)
    
    # Biến đổi log cho mục tiêu nếu bật
    if CONFIG['log_transform_target']:
        y = np.log1p(y)
    
    return X, y, df

# Hàm tăng cường dữ liệu bằng nhiễu Gaussian
def augment_data(X, y, noise_factor=0.01, num_copies=2):
    if not CONFIG['apply_gaussian_noise']:
        return X, y
    
    X_aug = X.copy()
    y_aug = y.copy()
    
    for _ in range(num_copies):
        noise = np.random.normal(loc=0, scale=noise_factor, size=X.shape)
        X_noisy = X + noise
        X_aug = np.vstack((X_aug, X_noisy))
        y_aug = np.hstack((y_aug, y))
    
    return X_aug, y_aug

# Hàm reshape dữ liệu cho BiLSTM
def reshape_for_bilstm(X, timesteps=1):
    # Reshape dữ liệu thành [samples, timesteps, features]
    n_samples, n_features = X.shape
    n_timesteps = timesteps
    n_new_samples = n_samples // n_timesteps
    X_reshaped = X[:n_new_samples * n_timesteps].reshape(n_new_samples, n_timesteps, n_features)
    return X_reshaped

# Hàm xây dựng mô hình BiLSTM
def build_bilstm_model(input_shape, lstm_units=16, dense_units=8, l2_reg=0.01, dropout_rate=0.3, learning_rate=0.001):
    l2_reg = max(l2_reg, 0.001)
    model = Sequential([
        Input(shape=input_shape),
        Bidirectional(LSTM(lstm_units, activation='tanh', return_sequences=False, kernel_regularizer=l2(l2_reg))),
        BatchNormalization(),
        Dropout(dropout_rate),
        Dense(dense_units, activation='relu', kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        Dropout(dropout_rate),
        Dense(1, activation='linear')
    ])
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=tf.keras.losses.Huber(), metrics=['mae'])
    return model

# Không gian siêu tham số cho BiLSTM
param_bounds = {
    'lstm_units': (8, 16),  
    'dense_units': (4, 16),
    'l2_reg': (0.001, 0.05),
    'dropout_rate': (0.2, 0.4),
    'learning_rate': (1e-4, 1e-2),
    'batch_size': (4, 16), 
    'epochs': (50, 100)
}

# Hàm mã hóa & giải mã particle
def random_particle():
    return np.array([
        np.random.randint(param_bounds['lstm_units'][0], param_bounds['lstm_units'][1] + 1),
        np.random.randint(param_bounds['dense_units'][0], param_bounds['dense_units'][1] + 1),
        np.random.uniform(param_bounds['l2_reg'][0], param_bounds['l2_reg'][1]),
        np.random.uniform(param_bounds['dropout_rate'][0], param_bounds['dropout_rate'][1]),
        np.random.uniform(param_bounds['learning_rate'][0], param_bounds['learning_rate'][1]),
        np.random.randint(param_bounds['batch_size'][0], param_bounds['batch_size'][1] + 1),
        np.random.randint(param_bounds['epochs'][0], param_bounds['epochs'][1] + 1)
    ])

def decode_particle(particle):
    params = {
        'lstm_units': int(particle[0]),
        'dense_units': int(particle[1]),
        'l2_reg': particle[2],
        'dropout_rate': particle[3],
        'learning_rate': particle[4],
        'batch_size': int(particle[5]),
        'epochs': int(particle[6])
    }
    params['l2_reg'] = max(params['l2_reg'], 0.001)
    return params

# Hàm fitness cho PSO
def fitness_function(particle, X_train, y_train):
    params = decode_particle(particle)
    model = build_bilstm_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        **{k: v for k, v in params.items() if k not in ['batch_size', 'epochs']}
    )
    
    kf = KFold(n_splits=CONFIG['k_folds'], shuffle=True, random_state=42)
    rmse_scores = []
    
    for train_idx, val_idx in kf.split(X_train):
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]
        
        early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
        
        model.fit(X_tr, y_tr, epochs=params['epochs'], batch_size=params['batch_size'],
                  validation_split=0.2, verbose=0, callbacks=[early_stopping, reduce_lr])
        y_pred = model.predict(X_val, verbose=0).flatten()
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        rmse_scores.append(rmse)
    
    return np.mean(rmse_scores)

# Hàm chạy PSO
def run_pso_bilstm(X_train, y_train, num_particles=10, max_iter=10):
    dim = len(param_bounds)
    bounds_array = np.array(list(param_bounds.values()))
    
    particles = [random_particle() for _ in range(num_particles)]
    velocities = [np.zeros(dim) for _ in range(num_particles)]
    p_best_positions = particles.copy()
    p_best_scores = [fitness_function(p, X_train, y_train) for p in particles]
    
    g_best_index = np.argmin(p_best_scores)
    g_best_position = p_best_positions[g_best_index]
    g_best_score = p_best_scores[g_best_index]
    
    w, c1, c2 = 0.5, 1.5, 1.5
    
    for iter in range(max_iter):
        print(f"\n🔁 Iteration {iter + 1}/{max_iter}")
        for i in range(num_particles):
            r1 = np.random.rand(dim)
            r2 = np.random.rand(dim)
            
            velocities[i] = (
                w * velocities[i]
                + c1 * r1 * (p_best_positions[i] - particles[i])
                + c2 * r2 * (g_best_position - particles[i])
            )
            
            particles[i] += velocities[i]
            particles[i] = np.clip(particles[i], bounds_array[:, 0], bounds_array[:, 1])
            particles[i][2] = max(particles[i][2], param_bounds['l2_reg'][0])
            
            score = fitness_function(particles[i], X_train, y_train)
            
            if score < p_best_scores[i]:
                p_best_scores[i] = score
                p_best_positions[i] = particles[i]
                
            if score < g_best_score:
                g_best_score = score
                g_best_position = particles[i]
                print(f"✅ Cập nhật g_best: Score = {g_best_score:.4f}")
    
    return g_best_position, g_best_score

# Hàm huấn luyện và đánh giá mô hình tối ưu
def train_and_evaluate(X_train, X_test, y_train, y_test, best_params):
    model = build_bilstm_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        **{k: v for k, v in best_params.items() if k not in ['batch_size', 'epochs']}
    )
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
    
    kf = KFold(n_splits=CONFIG['k_folds'], shuffle=True, random_state=42)
    metrics_all = []
    history_all = {'loss': [], 'val_loss': []}
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
        print(f"\n📂 Fold {fold + 1}/{CONFIG['k_folds']}")
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]
        
        history = model.fit(
            X_tr, y_tr, epochs=best_params['epochs'], batch_size=best_params['batch_size'],
            validation_split=0.2, verbose=0, callbacks=[early_stopping, reduce_lr]
        )
        y_pred = model.predict(X_val, verbose=0).flatten()
        metrics = calculate_metrics(y_val, y_pred)
        metrics_all.append(metrics)
        print(f"✅ Fold {fold + 1} RMSE: {metrics['rmse']:.4f}")
        
        history_all['loss'].append(history.history['loss'])
        history_all['val_loss'].append(history.history['val_loss'])
    
    # Đánh giá trên tập test
    y_pred_test = model.predict(X_test, verbose=0).flatten()
    test_metrics = calculate_metrics(y_test, y_pred_test)
    
    # Tính trung bình lịch sử huấn luyện
    max_len = max(len(h) for h in history_all['loss'])
    loss_avg = np.mean([np.pad(h, (0, max_len - len(h)), 'constant', constant_values=np.nan) for h in history_all['loss']], axis=0)
    val_loss_avg = np.mean([np.pad(h, (0, max_len - len(h)), 'constant', constant_values=np.nan) for h in history_all['val_loss']], axis=0)
    
    return model, test_metrics, metrics_all, loss_avg, val_loss_avg, y_pred_test

# Hàm trực quan hóa kết quả
def visualize_results(dataset_name, y_test, y_pred, loss_avg, val_loss_avg):
    if not CONFIG['visualize_results']:
        return
    
    plt.figure(figsize=(15, 12))
    
    # Loss trung bình
    plt.subplot(2, 2, 1)
    plt.plot(loss_avg, label='Training Loss')
    plt.plot(val_loss_avg, label='Validation Loss')
    plt.title('Average Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Huber Loss')
    plt.legend()
    
    # Predicted vs Actual
    plt.subplot(2, 2, 2)
    plt.scatter(y_test, y_pred, alpha=0.5)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    plt.title('Predicted vs Actual Effort')
    plt.xlabel('Actual Effort')
    plt.ylabel('Predicted Effort')
    
    # Error Distribution
    errors = y_test - y_pred
    plt.subplot(2, 2, 3)
    sns.histplot(errors, kde=True)
    plt.title('Error Distribution')
    plt.xlabel('Prediction Error')
    plt.ylabel('Frequency')
    
    plt.tight_layout()
    os.makedirs('visualizations', exist_ok=True)
    plt.savefig(f'visualizations/{dataset_name}_bilstm_results.png')
    plt.close()

# Hàm chính để chạy pipeline
def run_pipeline(dataset_config):
    dataset_name = dataset_config['name']
    print(f"\n🚀 Chạy pipeline cho dataset: {dataset_name}")
    
    # Đọc và tiền xử lý dữ liệu
    X, y, df = load_and_preprocess_data(
        dataset_config['path'],
        dataset_config['feature_columns'],
        dataset_config['target_column'],
        dataset_config['numeric_columns']
    )
    
    # Tăng cường dữ liệu
    X_aug, y_aug = augment_data(X, y, CONFIG['noise_factor'], CONFIG['num_noise_copies'])
    
    # Reshape dữ liệu cho BiLSTM
    X_aug = reshape_for_bilstm(X_aug, CONFIG['timesteps'])
    y_aug = y_aug[:X_aug.shape[0]]  # Cắt y để khớp số mẫu sau reshape
    
    # Chia tập train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X_aug, y_aug, test_size=CONFIG['test_size'], random_state=42
    )
    
    # Chạy PSO
    print("🔍 Tìm siêu tham số tối ưu bằng PSO...")
    best_particle, best_score = run_pso_bilstm(X_train, y_train, num_particles=10, max_iter=10)
    best_params = decode_particle(best_particle)
    print(f"🏆 Siêu tham số tốt nhất: {best_params}")
    print(f"📉 Score tốt nhất: {best_score:.4f}")
    
    # Huấn luyện và đánh giá
    model, test_metrics, metrics_all, loss_avg, val_loss_avg, y_pred_test = train_and_evaluate(
        X_train, X_test, y_train, y_test, best_params
    )
    
    # In kết quả
    print("\n📈 Kết quả đánh giá trên tập test:")
    for metric, value in test_metrics.items():
        print(f"📌 {metric.upper():<7}: {value:.4f}" if not np.isnan(value) else f"📌 {metric.upper():<7}: NaN")
    
    # Lưu kết quả
    if CONFIG['save_results']:
        results_df = pd.DataFrame([test_metrics])
        os.makedirs('results', exist_ok=True)
        results_df.to_csv(f'results/{dataset_name}_bilstm_results.csv', index=False)
        print(f"\nĐã lưu kết quả vào 'results/{dataset_name}_bilstm_results.csv'")
    
    # Lưu mô hình
    if CONFIG['save_model']:
        os.makedirs('models', exist_ok=True)
        model.save(f'models/{dataset_name}_bilstm_model.keras')
        print(f"Đã lưu mô hình vào 'models/{dataset_name}_bilstm_model.keras'")
    
    # Trực quan hóa
    visualize_results(dataset_name, y_test, y_pred_test, loss_avg, val_loss_avg)
    print(f"Đã lưu hình ảnh trực quan hóa vào 'visualizations/{dataset_name}_bilstm_results.png'")

# Cấu hình các dataset
datasets = [
    {
        'name': 'desharnais',
        'path': 'desharnais1.1_processed_corrected.csv',
        'feature_columns': [
            'TeamExp', 'ManagerExp', 'YearEnd', 'Length', 'Transactions', 'Entities',
            'Adjustment', 'PointsAjust', 'StartYear', 'ProjectDurationYears',
            'Transactions_Entities', 'Effort_PointsAjust', 'Effort_per_PointsAjust',
            'Transactions_per_Entities', 'Language_b\'1\'', 'Language_b\'2\'',
            'Language_b\'3\'', 'HighEffort', 'HighPointsAjust'
        ],
        'target_column': 'Effort',
        'numeric_columns': [
            'TeamExp', 'ManagerExp', 'YearEnd', 'Length', 'Transactions', 'Entities',
            'Adjustment', 'PointsAjust', 'StartYear', 'ProjectDurationYears',
            'Transactions_Entities', 'Effort_PointsAjust', 'Effort_per_PointsAjust',
            'Transactions_per_Entities', 'Language_b\'1\'', 'Language_b\'2\'',
            'Language_b\'3\'', 'HighEffort', 'HighPointsAjust'
        ]
    }
]

# Chạy pipeline cho từng dataset
if __name__ == "__main__":
    for dataset in datasets:
        run_pipeline(dataset)


🚀 Chạy pipeline cho dataset: desharnais
🔍 Tìm siêu tham số tối ưu bằng PSO...

🔁 Iteration 1/10
✅ Cập nhật g_best: Score = 0.1341

🔁 Iteration 2/10

🔁 Iteration 3/10
✅ Cập nhật g_best: Score = 0.1324

🔁 Iteration 4/10

🔁 Iteration 5/10

🔁 Iteration 6/10

🔁 Iteration 7/10

🔁 Iteration 8/10

🔁 Iteration 9/10

🔁 Iteration 10/10
🏆 Siêu tham số tốt nhất: {'lstm_units': 10, 'dense_units': 1, 'l2_reg': np.float64(0.018236164417873006), 'dropout_rate': np.float64(0.32471572716955666), 'learning_rate': np.float64(0.005911359310043052), 'batch_size': 13, 'epochs': 112}
📉 Score tốt nhất: 0.1324

📂 Fold 1/3
✅ Fold 1 RMSE: 0.3708

📂 Fold 2/3
✅ Fold 2 RMSE: 0.3822

📂 Fold 3/3
✅ Fold 3 RMSE: 0.2721

📈 Kết quả đánh giá trên tập test:
📌 MAE    : 0.3196
📌 MSE    : 0.1606
📌 RMSE   : 0.4008
📌 R2     : 0.8867
📌 MMRE   : 2.4827
📌 MDMRE  : 0.3384
📌 PRED25 : 29.4118

Đã lưu kết quả vào 'results/desharnais_bilstm_results.csv'
Đã lưu mô hình vào 'models/desharnais_bilstm_model.keras'
Đã lưu hình ảnh trực quan 